### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [1]:
!pip install langchain

In [2]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [3]:
!pip install langchain_groq

In [4]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x1169dd280>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x1169ddd00>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
!pip install langchain_core

In [8]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="I love you")
]

result=model.invoke(messages)

In [9]:
result

AIMessage(content="Je t'aime \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 19, 'total_tokens': 27, 'completion_time': 0.014545455, 'prompt_time': 8.884e-05, 'queue_time': 0.013941568, 'total_time': 0.014634295}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3982f39a-077a-4667-a8e2-cc5d57fc0136-0', usage_metadata={'input_tokens': 19, 'output_tokens': 8, 'total_tokens': 27})

In [10]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

"Je t'aime \n"

In [11]:
### Using LCEL- chain the components models and the stringoutparser abover
chain=model|parser
chain.invoke(messages)

"Je t'aime \n"

In [ ]:
### instead of just passing the messages , if i want to combine user input and application logic and then pass it to the llm , i use promopttemplates

In [13]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)



In [14]:
result=prompt.invoke({"language":"French","text":"Hello"})  # from above prompt , first param is language and second is user text

In [15]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [16]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'

In [25]:
!pip install streamlit